In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from tensorflow.python.keras import models, layers, optimizers
# import tensorflow
# from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
# from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re

%matplotlib inline

In [ ]:
import gzip

# with gzip.open('dataset/amazon_reviews_us_Books_v1_02.tsv.gz','r') as fin:        
#     for line in fin:        
#         print('got line', line)

In [ ]:
# import gzip
# import io
# gz = gzip.open('dataset/amazon_reviews_us_Watches_v1_00.tsv.gz', 'rb')
# f = io.BufferedReader(gz)
# counter = 0

# for line in f.readlines():
#     print(line)
#     if counter > 20:
#         break
#     counter=counter+1
            
# gz.close()

In [ ]:
import pandas as pd

# df = pd.read_csv('dataset/amazon_reviews_us_Watches_v1_00.tsv.gz', sep='\t', header=0)
df = pd.read_csv('dataset/amazon_reviews_us_Watches_v1_00.tsv', sep='\t', error_bad_lines=False)

In [ ]:
df.head(4)
# df.vine.value_counts()


In [ ]:
df[df['star_rating'] > 3]

In [ ]:
df['review_body'][df['star_rating'] > 3]

In [20]:
df.isna().sum()

marketplace          0
customer_id          0
review_id            0
product_id           0
product_parent       0
product_title        0
product_category     0
star_rating          0
helpful_votes        0
total_votes          0
vine                 0
verified_purchase    0
review_headline      0
review_body          1
review_date          0
all_review           1
clean_all_review     1
all_review_para      1
dtype: int64

In [21]:
df = df.where(pd.notnull(df), "")

### Combine Review Body & Head

In [ ]:
df['all_review'] = df['review_headline']+" "+df['review_body']

In [ ]:
# Unused

# emoticon_repl = {
#     # positive emoticons
#     r":-?d+": " good ", # :D, :-D, :DD, ecc. in lower case
#     r":[- ]?\)+": " good ", # :-), :), :-)), :)), ecc
#     r";-?\)+": " good ", # ;). ;)). ;-), ecc.
#     r"\(+-?:": " good ", # (:, (-:, ecc.
#     r"=\)+" : " good ",
#     r"\b<3\b" : " good ",    
#     # negative emoticons
#     r"[\s\r\t\n]+:/+": " bad ", # :/
#     r":\\+": " bad ", # :\
#     r"[\s\r\t\n]+\)-?:": " bad ",  # ):, )):, )-:, ecc.
#     r":-?\(+": " bad ", # :(, :-(, :((, ecc.
#     r"[\s\t\r\n]+d+-?:": " bad "
# }

In [ ]:
from collections import defaultdict
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [ ]:
def preprocess(sent):
    
    sent = str(sent).lower()
    sent = re.sub(r'^<div id="video.*>&nbsp;', '', sent) # Video-review part
    sent = re.sub('https?://[A-Za-z0-9./]+', '', sent) # URLs
    
    # Remove HTML Tag
    sent = re.sub('<.*?>', '', sent)    
    
    # Remove everything in bracket    
    sent = re.sub('\[[^)]*\]', '', sent)    
    
#     for k in emoticon_repl:
#         sent = re.sub(k, emoticon_repl[k], sent)
    
    sent = re.sub('[/]+', ' ', sent) # word1/word2 to word1 word2
    # Remove non-alphanumeric characters (but not - and _, might be useful for tokenization)
    sent = re.sub('[^A-Za-z0-9-_ ]+', '', sent)

    # Remove words that are digits only
    sent = re.sub('\b\d+\b', '', sent)                
    
    return sent

###  Apply Preprocess

In [ ]:
df['all_review'] = df['all_review'].apply(preprocess)

In [ ]:
df.head(5)

### Remove Stopwords

In [ ]:
len_df = len(df)
# print("%.7f" % p +" yeye")

In [ ]:
# stopwords
start = 400540
end = 500000

for entry in df['all_review'][start:end]:            
    res = [word for word in word_tokenize(entry) if not word in stopwords.words('english')]    
    res = (" ").join(res)  
    p = (start/len_df)*100
    print("%.7f" % p +" - "+str(start))
    df.loc[start,'clean_all_review'] = res
    if start >= end:
        break
    start = start+1
    
df.to_csv('amazon_reviews_us_Watches_vclean.csv')

In [ ]:
def test(x):
#     res = x + x
    data = []
    for i in x:
        data.append(i)
    return data
#         print(data.append(i))
        

In [ ]:
al = [1,2,4,6,7]
print(test(al))

# for x in df['all_review']:
#     print(x)

In [6]:
import time

### Paralel Looping

In [25]:
df.isna().sum()

marketplace          0
customer_id          0
review_id            0
product_id           0
product_parent       0
product_title        0
product_category     0
star_rating          0
helpful_votes        0
total_votes          0
vine                 0
verified_purchase    0
review_headline      0
review_body          0
review_date          0
all_review           0
clean_all_review     0
dtype: int64

In [24]:
df = df.where(pd.notnull(df), "")

In [ ]:
# Parallel Looping
start = 400500
end = 400550
# 500000

def remove_stopwords(df):
    for entry in df['all_review'][start:end]:            
        res = [word for word in word_tokenize(entry) if not word in stopwords.words('english')]    
        res = (" ").join(res)  
        p = (start/len_df)*100
        print("%.7f" % p +" - "+str(start))
        df.loc[start,'clean_all_review'] = res
        if start >= end:
            break
        start = start+1

        df.to_csv('amazon_reviews_us_Watches_vclean.csv')

In [83]:
import multiprocessing as mp
from multiprocessing import Pool
import remove_stopwords
from itertools import product

start = 0
end = 50

if __name__ == '__main__':
    p = mp.Pool(mp.cpu_count()) # Data parallelism Object
    df['clean_all_review'] = p.map(remove_stopwords.remove_sw, df['all_review'])
#     df['clean_all_review'] = df2['clean_all_review']

TypeError: expected string or bytes-like object

In [78]:
df[['review_headline','review_body','all_review','clean_all_review']][50:60]
# df2 = df[90:100].copy()
# df2[['review_headline','review_body','all_review','clean_all_review']]

KeyError: "['clean_all_review'] not in index"

In [ ]:
# df.to_csv('amazon_reviews_us_Watches_vclean.csv')
# df[['all_review','clean_all_review']][end-5:end+5]

# df[['all_review','clean_all_review']][497387-5:497387+5]
df.iloc[497388]

### Reload Preprocessed Data

In [80]:
df = pd.read_csv('amazon_reviews_us_Watches_vclean.csv', index_col=0, low_memory=False)

In [82]:
df=df[:10000]

In [1]:
df

NameError: name 'df' is not defined

In [76]:
df = df.drop('clean_all_review', axis=1)

In [ ]:
X=df.all_review.values
y=df.star_rating > 3

def check_presence(regex, stop = 1, verbose = True):
    j = 0
    tv = False
    for i in range(len(X)):
        if re.search(regex, str(X[i]).lower()):
            tv = True
            j += 1
            sentiment = "positive" if y[i] else "negative"
            if verbose:
                print("Review #",i)
                print("Sentiment: {}".format(sentiment))
                print("Occurrences of the regex {} :".format(regex), re.findall(regex, X[i].lower()))
                print("Review text:", X[i]+"\n")
            if j == stop:
                break
    return tv

In [ ]:
check_presence(r":-?d+")

In [ ]:
preprocess(X[1249])
# X[1249]

### Set Positive Sentiments

In [ ]:
df["positive"] = df.star_rating > 3

# Temp

In [23]:
# X[1249]
print(df['review_body'][1249])
print("")
print(df['all_review'][1249])
# print(df['clean_all_review'][1249])

KeyError: 1249

In [ ]:
df['total_votes'].value_counts()

In [ ]:
df = df.drop('clean_all_review', axis=1)

In [99]:
df = df2.copy()
df.loc[4:,'clean_all_review'] = df2['clean_all_review'][:10]

In [100]:
df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,all_review,clean_all_review
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31,five stars absolutely love this watch get comp...,five stars absolutely love watch get complimen...
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31,i love thiswatch it keeps time wonderfully i l...,love thiswatch keeps time wonderfully love wat...
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31,two stars scratches,two stars scratches
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31,five stars it works well on me however i found...,five stars works well however found cheaper pr...
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31,beautiful face but cheap sounding links beauti...,beautiful face cheap sounding links beautiful ...
5,US,6576411,RA51CP8TR5A2L,B00EYSOSE8,230493695,Casio Men's GW-9400BJ-1JF G-Shock Master of G ...,Watches,5,0,0,N,Y,No complaints,"i love this watch for my purpose, about the pe...",2015-08-31,no complaints i love this watch for my purpose...,complaints love watch purpose people complaini...
6,US,11811565,RB2Q7DLDN6TH6,B00WM0QA3M,549298279,Fossil Women's ES3851 Urban Traveler Multifunc...,Watches,5,1,1,N,Y,Five Stars,"for my wife and she loved it, looks great and ...",2015-08-31,five stars for my wife and she loved it looks ...,five stars wife loved looks great great price
7,US,49401598,R2RHFJV0UYBK3Y,B00A4EYBR0,844009113,INFANTRY Mens Night Vision Analog Quartz Wrist...,Watches,1,1,5,N,N,I was about to buy this thinking it was a ...,I was about to buy this thinking it was a Swis...,2015-08-31,i was about to buy this thinking it was a i w...,buy thinking buy thinking swiss army infantry ...
8,US,45925069,R2Z6JOQ94LFHEP,B00MAMPGGE,263720892,G-Shock Men's Grey Sport Watch,Watches,5,1,2,N,Y,Perfect watch!,Watch is perfect. Rugged with the metal &#34;B...,2015-08-31,perfect watch watch is perfect rugged with the...,perfect watch watch perfect rugged metal 34bul...
9,US,44751341,RX27XIIWY5JPB,B004LBPB7Q,124278407,Heiden Quad Watch Winder in Black Leather,Watches,4,0,0,N,Y,Great quality and build,Great quality and build.<br />The motors are r...,2015-08-31,great quality and build great quality and buil...,great quality build great quality buildthe mot...
